In [1]:
!pip install krwordrank

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import krwordrank
import pandas as pd
import re

data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/menupan_crawling.csv')
data.head()

,Unnamed: 0,res_name,res_type,res_address,res_homepage,res_score,res_theme,res_hour,res_room,smoking,amenity,parking,res_info,review,rev_date,rev_score
0,0,레빠쥬(les pages),양식,서울 강남구 신사동 506 강남출판문화센터 1층,http://www.lespages.co.kr/,3.8,"술 못마시는 친구와 함께, 일상데이트, 점심식사, 야외테라스, 스테이크, 와인, 양...",11:30 ~ 22:00BreakTime: 15:30~17:00,50석 / 방: 방없음실내 36석 / 테라스 16석,모두 금연석,Wifi,"4대, 주차방법: 무료, 주차장소: 업소앞/건물 지하주차장 (1시간 무료이용)","가로수길 뒷편에 자리잡은 피자, 파스타, 샌드위치를 판매하는 이탈리안 가정식 레스토...",파스타가 괜찮은 음식점@@\n감사합니다. 추천할께요^^@@\n관자의 쫄길함이 살아있...,2014/04/09\n2013/12/11\n2015/07/27\n2015/03/17...,3.0\n5.0\n3.0\n4.0\n5.0\n3.0\n3.0\n3.0\n5.0\n4.0
1,1,듀크렘(DEUX CREMES),카페/주점-카페,서울 강남구 신사동 545-4,Nan,3.9,"술 못마시는 친구와 함께, 외국 현지 음식(for 외국인), 소개팅, 일상데이트, ...",11:00 ~ 23:00,100석 / 방: 방없음,모두 금연석,Wifi,"100대, 주차방법: 유료, 주차장소: 발렛파킹/발렛비 : 2,000원",가로수길에서 타르트 맛집으로 이미 핫플레이스인 듀크렘.말 그대로 두가지 크림이라는 ...,직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들@@\n타르트...,2014/04/02\n2014/04/25\n2015/04/11\n2014/04/30...,1.3\n3.5\n4.0\n4.5\n4.5\n3.0\n2.5\n1.3\n4.0\n4...
2,2,팬아시아 신사가로수길점,기타/세계-동남아음식,서울 강남구 신사동 546-23 2층,Nan,4.1,"가족외식, 술 마시는 친구와 함께, 일상데이트, 브런치, 와인, 아시안 면요리, 타...",12:00 ~ 23:00,72석 / 방: 방없음,모두 금연석,Wifi,주차안됨,'잊을수 없는 경험' 이라는 신조가 팬아시아 레스토랑의 고객에 대한 마음가짐이다.팬...,보기에도 한번 가보고 싶은 곳이네요 ???@@\n동남아음식도 좋네요''''ㅎㅎ@@\...,2018/10/25\n2015/03/31\n2014/07/14\n2014/06/11...,4.5\n4.0\n5.0\n3.0\n4.5\n5.0\n3.5\n4.0\n5.0\n4...
3,3,더가든키친(The Garden Kitchen),호텔뷔페,서울 서초구 잠원동 6-1 더리버사이드호텔 1층,http://www.riversidehotel.co.kr,4.3,"가족외식, 돌잔치/회갑/생일, 어른과 함께라면, 아이와 함께라면, 일상데이트, 특별...",11:30 ~ 21:30,600석 / 방: 13개,모두 금연석,"놀이방,Wifi","200대, 주차방법: 유료, 주차장소: 발렛파킹","<리버사이드 더가든키친 할인이벤트>주중점심 53,200 ->39,900(25%)주중...",기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요^0^...,2015/10/01\n2017/02/13\n2016/10/11\n2015/12/16...,4.3\n4.5\n4.8\n4.0\n4.3\n4.3\n3.8\n4.0\n4.8\n4.5
4,4,시리얼고메(Serial Gourmet),양식,서울 강남구 신사동 541-13 2층,http://blog.naver.com/gourmet01,3.4,"소개팅, 특별한 날엔, 브런치, 주차공간, 스테이크, 맥주, 와인, 칵테일, 양식 ...",11:00 ~ 22:00일 11시~21시30분 / 브레이크 타임 없음,60석,모두 금연석,Nan,"10대, 주차방법: 유료, 주차장소: 발렛파킹/3,000원(2시간)","스타셰프 맛집 : 레이먼킴 쉐프 (오늘뭐먹지, 올리브쇼)시리얼고메는 유명 쉐프 레이...",맛도 그닥...서비스는 최악이더군요@@\n친구랑 같이 브런치 먹었는데 분위기도 좋고...,2016/04/02\n2015/12/18\n2016/03/14\n2015/04/08...,1.0\n4.0\n1.0\n4.0\n5.0\n5.0\n3.0\n4.0\n3.0\n4.3


In [4]:
data = data[['res_name', 'review']]
data.head()

,res_name,review
0,레빠쥬(les pages),파스타가 괜찮은 음식점@@\n감사합니다. 추천할께요^^@@\n관자의 쫄길함이 살아있...
1,듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들@@\n타르트...
2,팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 ???@@\n동남아음식도 좋네요''''ㅎㅎ@@\...
3,더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요^0^...
4,시리얼고메(Serial Gourmet),맛도 그닥...서비스는 최악이더군요@@\n친구랑 같이 브런치 먹었는데 분위기도 좋고...


In [5]:
data['review']

0       파스타가 괜찮은 음식점@@\n감사합니다. 추천할께요^^@@\n관자의 쫄길함이 살아있...
1       직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들@@\n타르트...
2       보기에도 한번 가보고 싶은 곳이네요 ???@@\n동남아음식도 좋네요''''ㅎㅎ@@\...
3       기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요^0^...
4       맛도 그닥...서비스는 최악이더군요@@\n친구랑 같이 브런치 먹었는데 분위기도 좋고...
                              ...                        
4172    소등심이 먹음직 스럽군요@@\n감사합니다. 추천할께요^^@@\n맛있어보입니다.......
4173    음료수도 무한리필이고 원두커피? 그것도 무한리필이더군요 양도 많아서 성인남자들이 가...
4174    냉모밀 진짜 여름에 짱짱이고요 사장님 특제소스랄까? 그런거 여서 진짜 맛있네요 여름...
4175    잘보고갑니다요''''ㅎㅎ@@\n먹기에도 맛도 좋을것 갇네요 ???@@\n친구랑 둘이...
4176    보기에도 양식의 진미가 보이네요 ???@@\n분위기도 아기자기하고 카레가 느끼하지도...
Name: review, Length: 4177, dtype: object

In [6]:
k = 0
## 정규식 : 한글, 숫자만 추출
for i in data['review']:
  text = re.compile('[ㄱ-ㅎ|\d\ㅏ-ㅣ|가-힣]+|@@').findall(i)
  data.loc[k, 'review'] = ' '.join(text).strip()
  k+=1

In [7]:
## 리뷰없는 식당 삭제
print(len(data))
drop_index = data[data['review'] == ''].index
data = data.drop(drop_index)
print(len(data))

4177
4120


In [8]:
print(data['review'][0])

파스타가 괜찮은 음식점 @@ 감사합니다 추천할께요 @@ 관자의 쫄길함이 살아있다 @@ 이곳 파스타가 좋네요 ㅎㅎ @@ 편안한 분위기에 맛있는 집 @@ 이탈리안 가정식 레스토랑 @@ 고르곤졸라 크래마 먹고싶어요 @@ 알리오 올리오 먹고싶군요 @@ 감사합니다 추천합니다 @@ 클래식한 분위기 짱 @@


In [9]:
from krwordrank.word import KRWordRank
from krwordrank.sentence import make_vocab_score
from krwordrank.sentence import MaxScoreTokenizer
from krwordrank.sentence import keysentence

wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(data['review'][:100], beta, max_iter, num_keywords=50)

stopwords = {'타르트', '관람객', '너무', '정말', '진짜'}
vocab_score = make_vocab_score(keywords, stopwords, scaling=lambda x:1)
tokenizer = MaxScoreTokenizer(vocab_score)




scan vocabs ... 
num vocabs = 2021
done


In [10]:
tokenizer.tokenize('바삭바삭한 감자튀김 맛집은 어디인가요?')

['바삭바삭한', '감자튀김', '맛집은', '어디인가요?']

In [11]:
penalty = lambda x: 0 if 25 <= len(x) <= 80 else 1

sents = keysentence(
    vocab_score, data['review'][:100], tokenizer.tokenize,
    penalty=penalty,
    diversity=0.4,
    topk=10
)

In [12]:
## key sentences 추출
for sent in sents:
    print(sent)

재료좋은것좀쓰세요 요즘시대가 어느시댄데 음식가지고 장난칩니까 @@ 여기 짜장떡볶기가 맛있어요 즉석떡볶기 깔끔합니다 만두도 맛있구오 아래분 여기는 사이트 등록한지 몇일 안되서 리뷰가 없는것 같네요 글고 여기는 가로수길 떡볶기1등정도 할듯 서래마을도 있어요 @@ 증말 니네 장사 그렇게하지마세요 제발 귀좀 열고 주문한대로 포장좀 해서보내라 김빠진튀김으로 서비스흉내내지말고 죽같이 만들어보내놓고는 밥맛없게 시간 다 말아먹고 @@ 지하라 좀 쾌적하지 못한 것 같음 공기가 좀 탁하고 환기가 잘 안됨 @@ ㅋㅋ완전웃겨 졸라드러어워서 도저히 못먹을것같은데 담배피고와서 그 손으로 조리하고있는데 위생상태 완전 최악이네요 그것도 모르고 맛있어서 가시는분들 정말 불쌍ㅎ 한번만더 보면 신고대상이네요 @@ 의자가 좀 불편해요 딱딱한 나무의자라서 테이블 신경쓰는 저는 약간 불편했네요 @@ 일반 즉석떡볶이집과 특별히 다를건 없어요 걍 주변에 즉석떡볶이집이 없으니 먹고싶을 때 가는 정도이지요 @@ 그냥 반장떡볶이로 시키면 사리가 이것저것 들어가있어서 좋아요 @@ 옛날 생각나게하는 떡볶이 그냥 옛날 추억에 잠길한 맛 정도 였던 것 같네요 @@ 줄 엄청 기다려서 먹었는데 음 즉떡이다 정도 ㅋㅋ 그래도 분위기랑 볶음밥이랑 같이 먹을 수 있어서 좋네요 ㅎㅎ @@ ㅎ음식도맛있고분위기도좋고무엇보다직원분들성격도좋고요리하시는남자분착하고웃을때기여웠어요ㅎ다먹을때눈너무마주쳐서쪽팔렷어요 @@ 짜장소스보다 고추장소스 좋아해서 항상 고추장소스로 먹어요 직원분들도 친절하시고 떡볶이 맛도 괜찮은 편이라 만족 @@ 감사합니다 추천합니다 @@ 다른 즉석떡볶이집들보다 양을 많이 주는 편인거 같아요 다른데가면 눈물나게 조금 줘서 ㅠ @@ 짜장떡볶이 있는 곳이 많지 않아서 갑니당ㅎ @@ 10월 23일 오픈한 오토웨이타워점 오픈주방인데 앞치마도 안하고 맨손으로 재료만지고 모자도 안쓰고 물묻은손 그냥 청바지에 쓱 닦고 한가해지니 나와서 담배 피시는데 거북했습니다 다른 떡볶이랑 비교되네요 @@ 제가 즉석떡볶이 정말 좋아하는데 여기 짜

In [13]:
# for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
#     print('%8s:\t%.4f' % (word, r))

In [14]:
# data.drop(['rev'], axis=1, inplace=True)

In [15]:
data['rev_list'] = 'a'
data.head()

,res_name,review,rev_list
0,레빠쥬(les pages),파스타가 괜찮은 음식점 @@ 감사합니다 추천할께요 @@ 관자의 쫄길함이 살아있다 @...,a
1,듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들 @@ 타르트...,a
2,팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 @@ 동남아음식도 좋네요 ㅎㅎ @@ 동남아음식에...,a
3,더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 0 ...,a
4,시리얼고메(Serial Gourmet),맛도 그닥 서비스는 최악이더군요 @@ 친구랑 같이 브런치 먹었는데 분위기도 좋고 맛...,a


In [16]:
type(data['review'])

pandas.core.series.Series

In [42]:
## 가게별 review 데이터 @@로 split해서 list로 변경
i = 0
for rev in data['review']: 
  data.loc[i,'rev_list'] = rev.split('@@')
  i+=1

In [43]:
data['rev_list'][-10:]

4072    [어제 동생넘이랑 감자탕 시켰는데 깻잎에서 무슨 벌레 껍데기라고 해야하나 하여튼 이...
4102    [포트오브모카 좋아요 ,  맛있어보입니다 ,  요즘 커피값이 장난 아니던데 여기는 ...
4103    [신입생일때부터 간 파전집이네요 ㅋㅋㅋㅋ 동기들기리 모이면 꼭 여기가는데 진짜 맛납...
4104    [가격이 저렴한 만큼 고기가 완전 맛있거나 하진 않다 그래도 무난한 고기맛인데 양도...
4107    [한번 가보고 싶은 곳이네요 ,  밥 먹고 오래 앉아있어도 싫은 내색 하나 안 하셔...
4108    [시원하고 부드러운 아메리카노 ,  과일상자 포인트 무용지물 ,  그렇게 넓지는 않...
4109    [레몬에이드가 좋아요 ㅎㅎ ,  본인을 매니저라 칭하는 안경쓰고 키작고 통통하신분 ...
4112    [진짜 진짜 죄송하지만 닭도리탕 맛있다고 해서 입소문듣고 갔는데요 정말 닭이랑 국물...
4113    [수요미식회 보고 갔었어요 제가 먹어본 제육볶음중에 짱 게다가 가격도 정말 착해요 ...
4119    [보기에도 양식의 진미가 보이네요 ,  분위기도 아기자기하고 카레가 느끼하지도 않고...
Name: rev_list, dtype: object

In [46]:
from krwordrank.sentence import summarize_with_sentences

keywords= []
sents = []
not_key = []

penalty = lambda x:0 if (25 <= len(x) <= 80) else 1
stopwords = {'타르트가', 'ㅋㅋ', '맛있', '정말', '진짜'} ## 제외하고픈 단어


def summarize(review):
  return summarize_with_sentences(review, penalty=penalty, stopwords = stopwords, diversity=0.4, num_keywords=1, num_keysents=1, scaling=lambda x:1, verbose=False)

## 가게별로 keyword, key_sentence 하나씩 뽑아서 리스트에 넣기
## 리뷰가 작은 가게는 추출이 안되는 문제가 있음
for i in range(len(data['rev_list'])):
  try:
    key, sentence = summarize(pd.Series(data['rev_list'][i]))
    keywords.append(key)
    sents.append(sentence)
  ## 추출 안되는 가게 인덱스 저장
  except:
    not_key.append(i)



In [47]:
## 리뷰를 가지고있는 가게 개수
len(data['rev_list'])

4175

In [49]:
## 리뷰에서 키워드 추출이 가능한 가게 개수
ok = []
for i in range(len(data['rev_list'])):
  if i not in not_key:
    ok.append(i)

len(ok) 

1149

In [51]:
## 가게명 + 키워드 + 메인리뷰 추출

for i in range(len(ok)):
  print('가게명 :', data.loc[ok[i],'res_name'],'키워드 :', keywords[i],'메인리뷰 :', sents[i])

가게명 : 듀크렘(DEUX CREMES) 키워드 : {'타르트': 5.047375402433487} 메인리뷰 : [' 타르트가 촉촉 부드럽부드럽 주말 저녁에 갔더니 사람 바글바글 그래서 웨이팅도 좀 햇어요 ㅋㅋㅋㅋ 이젠 익숙하다 ']
가게명 : 팬아시아 신사가로수길점 키워드 : {'팟타이': 2.7506362812645735} 메인리뷰 : [' 친한 여자애들이 팟타이 먹고싶다고 난리난리 쳐서 갓엇는데 나쁘지 않더라구여 ']
가게명 : 더가든키친(The Garden Kitchen) 키워드 : {'음식': 1.0} 메인리뷰 : ['기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 0 ']
가게명 : 벤스쿠키(Ben's Cookie) 신사점 키워드 : {'쿠키': 5.319318364354846} 메인리뷰 : ['막 그 렇게 맛있는건 아닌데 다른 쿠키하고 맛이 좀 다르긴 다름 ㅋ ']
가게명 : 듀자미(Deux Amis) 키워드 : {'케익': 1.800411612961375} 메인리뷰 : ['확장해서 좋다 했더니 수요미식회 나오는바람에 한동안 또 못가겠네요 케익 맛있는데 차 종류가 너무 비싼것만 흠이라면 흠입니다 ']
가게명 : 르브런쉭(Le Brunchic) 키워드 : {'ㅎㅎ': 2.2081182097815124} 메인리뷰 : [' 맛있었어요 ㅎㅎ 음식 만족스럽고 ㅎㅎ 분위기 만족스럽고 ㅎㅎ 커피 한잔하면서 도란도란 얘기하기 좋네요 ㅎㅎ ']
가게명 : 반장즉석떡볶이 키워드 : {'떡볶이': 2.9500640775052482} 메인리뷰 : [' 일반 즉석떡볶이집과 특별히 다를건 없어요 걍 주변에 즉석떡볶이집이 없으니 먹고싶을 때 가는 정도이지요 ']
가게명 : 소공동뚝배기 압구정본점 키워드 : {'오삼직화': 1.864191065158388} 메인리뷰 : [' 처음에 엄청 비싼 가격인줄 알았는데 알고보니 오히려 저렴하더라구요 2인 가격이었으니까요 전 오삼직화가 가장 맛있었어요 ㅎㅎ ']
가게명 : 개화옥 가로수길점 키워드 : {'불